In [31]:
saprk.stop()

NameError: name 'saprk' is not defined

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Row
from pyspark.sql.window import Window
from pyspark.sql.types import NumericType
from graphframes import GraphFrame
from pyiceberg.catalog import load_catalog
import re

# --- 1. Spark с Iceberg + MinIO ---
spark = SparkSession.getActiveSession()
if spark is not None:
    print("✅ Подключен к существующей SparkSession")
else:
    print("⚠️ Активная сессия не найдена — создаю новую...")
    spark = (
        SparkSession.builder
            .appName("glazkov_type_rest")
            .getOrCreate()
    )
print("=" * 80)

# 🧠 Основные параметры кластера
print(f"Spark master:            {spark.sparkContext.master}")
print(f"Spark version:           {spark.version}")
print(f"App name:                {spark.sparkContext.appName}")
print(f"Executors (parallelism): {spark.sparkContext.defaultParallelism}")
print(f"Executor memory:         {spark.conf.get('spark.executor.memory', 'n/a')}")
print(f"Driver memory:           {spark.conf.get('spark.driver.memory', 'n/a')}")

print("-" * 80)

# 🧊 Iceberg / Catalog / Warehouse
print(f"Catalog type:            {spark.conf.get('spark.sql.catalog.ice.type', 'n/a')}")
print(f"Catalog URI:             {spark.conf.get('spark.sql.catalog.ice.uri', 'n/a')}")
print(f"Warehouse:               {spark.conf.get('spark.sql.catalog.ice.warehouse', 'n/a')}")
print(f"Default catalog:         {spark.conf.get('spark.sql.defaultCatalog', 'n/a')}")

print("-" * 80)

# 🪣 S3 / MinIO настройки
print(f"S3A endpoint:            {spark.conf.get('spark.hadoop.fs.s3a.endpoint', 'n/a')}")
print(f"S3A path style:          {spark.conf.get('spark.hadoop.fs.s3a.path.style.access', 'n/a')}")
print(f"S3A SSL enabled:         {spark.conf.get('spark.hadoop.fs.s3a.connection.ssl.enabled', 'n/a')}")
print(f"S3A provider:            {spark.conf.get('spark.hadoop.fs.s3a.aws.credentials.provider', 'n/a')}")

print("-" * 80)

# ⚙️ Iceberg расширения и классы
print(f"Iceberg extensions:      {spark.conf.get('spark.sql.extensions', 'n/a')}")
print(f"Iceberg catalog class:   {spark.conf.get('spark.sql.catalog.ice', 'n/a')}")

print("-" * 80)

# 🧾 Проверим доступность REST-каталога
try:
    from urllib.request import urlopen
    uri = spark.conf.get("spark.sql.catalog.ice.uri")
    resp = urlopen(uri + "/v1/config").read().decode("utf-8")
    print(f"REST catalog reachable ✅ {uri}")
except Exception as e:
    print(f"⚠️ REST catalog NOT reachable ({e})")

print("=" * 80)

# --- Подключаем PostgreSQL ---
# pg_url = "jdbc:postgresql://192.168.150.10:5432/MZ_DWH"
# pg_props = {
#     "user": "glazole",
#     "password": "MEbt7xr39yRhXUBj",
#     "driver": "org.postgresql.Driver"
# }

✅ Подключен к существующей SparkSession
Spark master:            spark://spark-master:7077
Spark version:           3.5.0
App name:                glazkov_type_rest
Executors (parallelism): 2
Executor memory:         2g
Driver memory:           3g
--------------------------------------------------------------------------------
Catalog type:            rest
Catalog URI:             http://iceberg-rest:8181
Warehouse:               s3a://warehouse/iceberg/
Default catalog:         ice
--------------------------------------------------------------------------------
S3A endpoint:            http://minio:9000
S3A path style:          true
S3A SSL enabled:         false
S3A provider:            n/a
--------------------------------------------------------------------------------
Iceberg extensions:      org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
Iceberg catalog class:   org.apache.iceberg.spark.SparkCatalog
---------------------------------------------------------------

In [18]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog("ice", uri="http://iceberg-rest:8181")

# Попробуем зарегистрировать вручную
catalog.create_namespace("bronze", properties={
    "location": "s3a://warehouse/iceberg/bronze"
})

In [19]:
spark.sql("SHOW NAMESPACES IN ice").show()

+---------+
|namespace|
+---------+
|   bronze|
+---------+



In [20]:
cat = spark.catalog.listDatabases()
print(cat)

[Database(name='bronze', catalog='ice', description=None, locationUri='s3a://warehouse/iceberg/bronze')]


In [28]:
import boto3
import re
import io

# === Подключение к MinIO ===
s3 = boto3.client(
    "s3",
    endpoint_url="http://minio:9000",  # если запускаешь внутри docker сети
    aws_access_key_id="minio",
    aws_secret_access_key="minio_minio",
    region_name="us-east-1",
)

bucket = "warehouse"
prefix = "iceberg/bronze/"

# === Список всех JSON-файлов в каталоге ===
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

while True:
    for obj in response.get("Contents", []):
        key = obj["Key"]
        if not key.endswith(".json"):
            continue

        body = s3.get_object(Bucket=bucket, Key=key)["Body"].read().decode("utf-8")
        if "warehouse.minio" in body:
            print(f"Fixing: {key}")
            fixed = re.sub(r"warehouse\.minio", "warehouse", body)
            s3.put_object(
                Bucket=bucket,
                Key=key,
                Body=io.BytesIO(fixed.encode("utf-8")),
                ContentType="application/json",
            )

    # --- Если есть следующая страница результатов (для больших бакетов) ---
    if response.get("IsTruncated"):
        response = s3.list_objects_v2(
            Bucket=bucket,
            Prefix=prefix,
            ContinuationToken=response["NextContinuationToken"],
        )
    else:
        break

print("✅ Все вхождения 'warehouse.minio' заменены на 'warehouse'")

✅ Все вхождения 'warehouse.minio' заменены на 'warehouse'


In [30]:
for obj in s3.list_objects_v2(Bucket=bucket, Prefix=prefix)["Contents"]:
    data = s3.get_object(Bucket=bucket, Key=obj["Key"])["Body"].read().decode("utf-8")
    if "warehouse.minio" in data:
        print("❌ Осталось:", obj["Key"])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 11: invalid start byte

In [29]:
tables = [
    ("crpt_2025_raw", "v31"),
    ("crpt_load_log", "v34"),
    ("f4_sprav_gtin", "v2"),
    ("f5_sprav_md", "v2"),
    ("f7_sprav_gtin_vac", "v2")
]

for table_name, version in tables:
    spark.sql(f"""
        CALL ice.system.register_table(
            'bronze.{table_name}',
            's3a://warehouse/iceberg/bronze/{table_name}/metadata/{version}.metadata.json'
        )
    """)
    print(f"✅ {table_name} зарегистрирована в REST-каталоге")

Py4JJavaError: An error occurred while calling o54.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:211)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:323)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:262)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:368)
	at o

In [4]:
cat = load_catalog("ice", uri="http://iceberg-rest:8181")
cat.list_tables("bronze")

NoSuchNamespaceError: NoSuchNamespaceException: Namespace does not exist: bronze

In [8]:
spark.stop()